L'objectif de ce Notebook est d'automatiser la production des reportings de matrice de transition 

# Authentification
Télécharger un fichier json de gcp service account pour s'identifier 
L'authentification se fait ici en chargeant les crédentials d'un fichier 
voir https://cloud.google.com/bigquery/docs/authentication/service-account-file

In [1]:
from google.cloud import bigquery

# Explicitly use service account credentials by specifying the private
# key file. All clients in google-cloud-python have this helper.
client = bigquery.Client.from_service_account_json(r"C:\Users\lveyssiere\Downloads\Equipe_DA_DWH_DA_Projects.json")

In [2]:
job_config = bigquery.QueryJobConfig()
job_config.use_legacy_sql = False
job_config.allow_large_results = True

# Requête
L'objet de la requête est de construire la matrice de transition 
Les paramètres de la requête sont le mois actuel, le pays et le mois précédent 

In [3]:
query_string = """

-- Liste des variables selectionnées 

    SELECT 
    LagN AS Previous_Date,
    N AS Current_Date,
    A.Company,
    #A.Country,        
    BranchDescription,
     
    
    LagBranch AS Branch_Previous_Month,
    
    CASE
    WHEN Branch is null THEN LagBranch
    ELSE Branch END AS Branch_Current_Month,
    
    CASE 
    WHEN DAOName is null THEN "Unknown"
    ELSE DAOName END AS DAOName,
    A.ClientNumber,
    ClientType,
    A.ClientName,
    A.ClientFirstName,LoanCycle,
    A.ContractNumber,
    Category,
    CategoryDesc,
    DisbursementDate,
    LoanDuration,
    
    MaturityDate,
    
    LagLoanStatus AS LoanStatus_Previous_Month,
    
    CASE WHEN LoanStatus is null THEN "LIQ"
    ELSE LoanStatus END AS LoanStatus_Current_Month,
    
    LagExposition AS GLP_Previous_Month,
    LagExposition/CurrencyRate AS GLP_Previous_Month_Euro,
    
    CASE WHEN Exposition is null THEN 0 
    ELSE Exposition END AS GLP_Current_Month,
    CASE WHEN Exposition is null THEN 0 
    ELSE Exposition/CurrencyRate END AS GLP_Current_Month_Euro,
    PreviousPrincipalDueAmt/CurrencyRate AS PreviousPrincipalDueAmt,
    PrincipalDueAmt/CurrencyRate AS PrincipalDueAmt,
    PreviousTotalDueAmt/CurrencyRate AS PreviousTotalDueAmt,
    TotalDueAmt/CurrencyRate AS TotalDueAmt,
    
    
    CASE
        WHEN (LagNbOverdueDays<1   OR LagNbOverdueDays is null) AND LagExposition>0 THEN "0- Healthy"
        WHEN LagNbOverdueDays>=1   AND LagNbOverdueDays<=30      AND LagExposition>0 THEN "1- %   1-30"
        WHEN LagNbOverdueDays>=31  AND LagNbOverdueDays<=60      AND LagExposition>0 THEN "2- %  31-60"
        WHEN LagNbOverdueDays>=61  AND LagNbOverdueDays<=90      AND LagExposition>0 THEN "3- %  61-90"
        WHEN LagNbOverdueDays>=91  AND LagNbOverdueDays<=120  AND LagExposition>0 THEN "4- %  91-120"
        WHEN LagNbOverdueDays>=121 AND LagNbOverdueDays<=150  AND LagExposition>0 THEN "5- % 121-150"
        WHEN LagNbOverdueDays>=151 AND LagNbOverdueDays<=180  AND LagExposition>0 THEN "6- % 151-180"
        WHEN LagNbOverdueDays>180  AND Exposition>0 THEN "7- % >180"
        WHEN LagExposition = 0 then "0- Close"
    ELSE "Unknown"
    END AS ParType_Previous_Month,

    CASE
        WHEN (NbOverdueDays<1 OR NbOverdueDays is null) AND Exposition>0 THEN "0- Healthy"
        WHEN NbOverdueDays>=1 AND NbOverdueDays<=30 AND Exposition>0    THEN "1- %   1-30"
        WHEN NbOverdueDays>=31 AND NbOverdueDays<=60 AND Exposition>0   THEN "2- %  31-60"
        WHEN NbOverdueDays>=61 AND NbOverdueDays<=90 AND Exposition>0   THEN "3- %  61-90"
        WHEN NbOverdueDays>=91 AND NbOverdueDays<=120 AND Exposition>0  THEN "4- %  91-120"
        WHEN NbOverdueDays>=121 AND NbOverdueDays<=150 AND Exposition>0 THEN "5- % 121-150"
        WHEN NbOverdueDays>=151 AND NbOverdueDays<=180 AND Exposition>0 THEN "6- % 151-180"
        WHEN NbOverdueDays>180 AND Exposition>0 THEN "7- % >180"
        WHEN (Exposition = 0 OR Exposition is null) 
             AND (LagNbOverdueDays<=150 OR LagNbOverdueDays is null) THEN "0- Closed"
        WHEN (Exposition = 0 OR Exposition is null) 
             AND LagNbOverdueDays>150 THEN "9- Written off"
    ELSE "Unknown"
    END AS ParType_Current_Month,

    LagNbOverdueDays AS NbOverdueDays_Previous_Month,
    
    CASE 
    WHEN NbOverdueDays is null THEN 0 
    ELSE NbOverdueDays END AS NbOverdueDays,
    
    CASE WHEN LagMaxOverdueDays is null THEN 0 
    ELSE MaxOverdueDays END AS MaxOverdueDays_Previous_Month,
    
    CASE 
    WHEN MaxOverdueDays is null THEN 0 
    ELSE MaxOverdueDays END AS MaxOverdueDays,
    
    LagMaxScheduleDate AS MaxScheduleDate_Previous_Month,
    
    CASE WHEN NextScheduleDate is null THEN "{moisN}"
    ELSE NextScheduleDate END AS NextScheduleDate,
    
    LagRestructuration AS Restructuration_Previous_Month,Restructuration,
    LagDisbursementAmount, DisbursementAmount,
    LagDisbursementAmount/CurrencyRate AS LagDisbursementAmountEuro, 
    DisbursementAmount/CurrencyRate AS DisbursementAmountEuro,
    LagDisbursed,Disbursed,
    LagDisbursed/CurrencyRate AS LagDisbursedEuro,
    Disbursed/CurrencyRate AS DisbursedEuro,B.Currency, CurrencyRate

-- Data are gathered from the mcr_loan consolidé 


FROM

-- Block A extrait les données pour le mois précédent 

(SELECT
"{moisLagN}" AS LagN,
ClientNo AS ClientNumber,AccountContractNo AS ContractNumber,ClientName,ClientFirstName,DAOName AS LagDAOName,Branch AS LagBranch,
LoanCycle,Category,CategoryDesc,
DisbursementDate,LoanDuration,LoanStatus AS LagLoanStatus,PrincipalOutstanding+TotalPrincipalDueAmt AS LagExposition,
TotalCurNoOfDaysOverdue AS LagNbOverdueDays,
MaxOverdueDays AS LagMaxOverdueDays,
NextScheduleDate AS LagMaxScheduleDate,
DisbursementAmount AS LagDisbursementAmount,
TotalPrincipalDueAmt AS PreviousPrincipalDueAmt,
TotalDueAmt AS PreviousTotalDueAmt,
Currency,DateData,

CASE 
  WHEN Extract(Month FROM DisbursementDate)=Extract(Month FROM DateData)
  AND  Extract(Year FROM DisbursementDate)=Extract(Year FROM DateData) THEN DisbursementAmount
ELSE 0 END AS LagDisbursed,

CASE 
WHEN LoanCycle=1 then "New Client"
ELSE "Renewed Client"
END AS ClientType,
Date(MaturityDate) AS MaturityDate,
RestructurationIndicator AS LagRestructuration,
Company,
CASE 
    WHEN Company="MCCD" THEN "MCCN"
    WHEN Company="MCNC" THEN "MCCN"
    ELSE Company
    END
AS Country

from `ConsolidatedFiles.mcr_loan`
        
        WHERE Date(DateData)="{moisLagN}") A

LEFT JOIN 

-- Block B extrait les données pour le mois actuel 

(SELECT
"{moisN}" AS N,
ClientNo AS ClientNumber,AccountContractNo AS ContractNumber,ClientName,ClientFirstName,DAOName,Branch,
LoanStatus,PrincipalOutstanding+TotalPrincipalDueAmt AS Exposition,
TotalCurNoOfDaysOverdue AS NbOverdueDays,
MaxOverdueDays AS MaxOverdueDays, 
NextScheduleDate AS NextScheduleDate,
DisbursementAmount, 
DateData,
TotalPrincipalDueAmt AS PrincipalDueAmt,
TotalDueAmt,

CASE 
  WHEN Extract(Month FROM DisbursementDate)=Extract(Month FROM DateData)
  AND  Extract(Year FROM DisbursementDate)=Extract(Year FROM DateData) THEN DisbursementAmount
ELSE 0 END AS Disbursed,
RestructurationIndicator AS Restructuration,
Company, 
CASE 
    WHEN Company="MCCD" THEN "MCCN"
    WHEN Company="MCNC" THEN "MCCN"
    ELSE Company
    END
AS Country,Currency

from `ConsolidatedFiles.mcr_loan`

WHERE Date(DateData)="{moisN}") B

-- la Jointure ce fait sur le numéro client, contrat pour chaque pays 

ON A.Company=B.Company AND A.ClientNumber=B.ClientNumber AND A.ContractNumber=B.ContractNumber

-- Ajout de la description des agences 

LEFT JOIN 

(SELECT Subsidiary, BranchID, MAX(BranchName) AS BranchDescription
    FROM Alpha_DWH.Branch_20190515_V2 WHERE ((BranchName <> "") OR (BranchName is not null))
    GROUP BY Subsidiary,BranchID) C

ON A.Company=C.Subsidiary AND A.LagBranch=C.BranchID

-- Ajout du taux de change TO DO 

LEFT JOIN 

 `group_ops.Currencyrates2` D

ON A.Company=D.Company AND A.Currency=D.Currency -- Obligé de prendre lag sinon pas d'info 
AND Extract(Month FROM A.DateData)=D.Month
AND Extract(Year FROM A.DateData)=D.Year

WHERE  LagExposition>0 
        AND CASE WHEN "{pays}"<> "Consolide" THEN A.Country="{pays}"
                 ELSE A.Company=A.Company END 

/* V1 Matrice de transition 
Axes d'amélioration 
- automatiser le block d'entrée avec le taux de change des différentes variables 


*/

"""


# Boucle 
La boucle automatise la creation des fichiers csv 

Définition de la période du calcul de la matrice de transition date de début de semaine et date de fin de semaine 

In [4]:
import datetime
jour = datetime.datetime.now() - datetime.timedelta(days=1)
datefin=jour.date()
print(datefin)
datedebut = jour - datetime.timedelta(days=7)
datedebut = datedebut.date()
print(datedebut)


2019-06-24
2019-06-17


Selection des pays 

In [13]:
datedebut="2019-06-01"
print(datedebut)
#datefin="2019-06-10"
list_pays=['MCMG','MCBF','MCNG','MCML','MCSN','MCTN','MCZW','MCDC','MCCN','MCCI','Consolide']

2019-06-01


Boucle for permettant le calcul de chacune des bases de données 

In [14]:
for countries in list_pays:
    print(countries)
    query_job = client.query(query_string.format(moisLagN = datedebut,moisN = datefin,pays=countries))
    raw_data = query_job.to_dataframe()
    raw_data.to_csv(r"C:\Users\lveyssiere\Desktop\Transition_{pays}.csv".format(pays=countries), index=False),
    

MCMG
MCBF
MCNG
MCML
MCSN
MCTN
MCZW
MCDC
MCCN
MCCI
Consolide


Envoi d'email 
Définition des personnes ? 

In [17]:
import smtplib
conn = smtplib.SMTP('smtp.gmail.com', 587)
conn.ehlo()

(250,
 b'smtp.gmail.com at your service, [178.20.52.56]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8')

In [18]:
conn.starttls()

(220, b'2.0.0 Ready to start TLS')

In [22]:
x=0.1
x+0.2==0.3

False

NameError: name 'Print' is not defined